### Simulador Imobiliário para Imóveis na Planta (Minha Casa Minha Vida)

Protótipo de simulador criado com as informações e taxas baseadas nas condições de financiamento para o Programa Minha Casa Minha Vida fornecidos pela CEF.

Os cálculos apresentados foram puramente baseados no conhecimento do autor e podem conter divergências com o simulador real, pois não levam em consideração cálculos de subsídio e nem bases de cálculo para recorte territorial. A base considerará o cálculo para grandes metrópoles, idade, valor da renda e desconto. A diferença maior está nas condições apresentadas com a taxa flutuante "TR"/"Taxa Referêncial", que consta no simulador a partir de um Web Scraping buscando o valor acumulado mais recente do mês anterior como base para atualização monetária e correções. Página 1.3.1 do Notebook.

A intenção em criar um teste de simulador de financiamento é apenas poder incluir as variações das taxas flutuantes no valor da parcela final, com o intuito de chegar mais perto do valor real. Embora hajam infinitas questões e variações de valores, local, correções, o conceito por trás deste projeto é puramente experimental e não tem qualquer interesse em inutilizar ou questionar os meios/ métodos utilizados atualmente.


~x~

In [ ]:
import pandas as pd
import numpy_financial as npf
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta


### Condições Fictícias

In [ ]:
# Perfil do Cliente

# 2 Participantes, casados e com dependentes.
# Renda Bruta familiar mensal de dois salários mínimos e meio: R$3530,00 (Base do Salário Mínimo: R$1412,00)
# Possuem juntos de FGTS para utilizar na aquisição do primeiro imóvel: R$8.000,00
# Não receberam desconto no imóvel por parte do construtor.
# A data de nascimento do mais velho entre os dois participantes é de: 06/02/1987
# Subsídio concedido pelo governo: R$4.254,00
# Desconto total: FGTS + Subsídio = R$12.254,00
# Valor do Imóvel:  R$190.000,00

### 0 - Preenchimento dos Valores da Operação

In [ ]:
# Informações dos Valores

valor_venda = float(input('Informe o Valor de Venda do Imóvel: '))
valor_renda_bruta = float(input('Informe o Valor da Renda do Cliente:' ))
desconto = float(input('Informe o valor TOTAL do desconto, caso haja (FGTS, Subsidio...): '))
data_de_nascimento_cliente = input('Informe a data de Nascimento do Cliente no formato (DD/MM/AAAA): ')

# Separar os valores da Data de Nascimento
dia = int(data_de_nascimento_cliente[0:2])
mes = int(data_de_nascimento_cliente[3:5])
ano = int(data_de_nascimento_cliente[6:])



### 1 - Idade do Cliente (Data de Nascimento)

In [ ]:
# Definir a Idade do Cliente

nascimento = datetime(ano, mes, dia) # Data de Nascimento
um_mes = timedelta(days=30) # Valor de um mês com 30 dias
hoje = datetime.today() # Data de Hoje
diferenca = hoje - nascimento # Calcular a diferença entre a Data de Hoje e o Nascimento para obter o valor da idade
total_dias = diferenca.days # Tranformar o valor da diferença para dias
idade = (total_dias / 365) # Dividir o valor do total em dias por 365 dias (1 ano).
idade = round(idade, 2) # Formatar para um valor com duas casas decimais

# Separar os valores de mês e ano para buscar o valor acumulado mensal da TR.

if hoje.month > 1:
    mes_tr = hoje.month -1
    ano_tr = str(hoje.year)
else:
    mes_tr = 12
    ano_tr = str(hoje.year - 1)

mes_tr_str = "{:02}".format(mes_tr)

#print(mes_tr_str, ano_tr)

### 1.1 -  Verificação dos Prazos de acordo com a Idade informada

In [ ]:
# Definir o prazo de financiamento do cliente

idade_max = 80.6 # Prazo máximo para cálculo de idade financiamento 80 anos e 6 meses são subtraídos da idade do cliente.
prazo_max = 360 # Prazo máximo de financiamento 360 meses (30 anos)

meses = int((idade_max * 12) - (idade * 12)) # Cacular o tempo em meses

# Definir as condições de prazo baseado na idade do cliente
if meses < prazo_max:
    meses = meses
    if meses < 120:
        meses = 0
        #print('Não é possível financiar')
    
    #print(meses)
else:
    meses = prazo_max
    #print(meses)



### 1.2 - Informações de valores dos Seguros DFI e MIP

In [ ]:
# Tabela base com os valores de alíquota do Seguro MIP (Morte por Invadez Permanente)

# Aliquota MIP
valor_aliquota = [0.000093, 0.000096, 0.000116,  0.000154,  
                  0.000252,  0.000386,  0.000676,  0.001533,  
                  0.002371,  0.003259,  0.004894,  0.005312] 

# Lista com os limites de idade referentes à alíquota
idade_mip = [26, 31, 36, 41, 46, 51, 56, 61, 66, 71, 76, 80.6]

# Alíquota base para o seguro de cobertura DFI (Danos Físicos ao Imóvel)
seguro_dfi = 0.000079

# Criar um DataFrame para visualizar as informações de seguro em uma Tabela.
seguro_mip_idades = {'Alíquota MIP' : valor_aliquota,
                    'Idade Máx' : idade_mip}

df_seguro_mip_idades = pd.DataFrame(seguro_mip_idades)

#df_seguro_mip_idades

### 1.3 - Definição das taxas de juros para utilizar na Operação de Financiamento

In [ ]:
# Tabela base de Taxas de juros para os perfis de renda inferiores à R$4.400,01

taxas_nominais = [0.05, 0.055, 0.06, 0.07, 0.081600] # Valores das Taxas Nominais definidas para os perfis de Renda
renda_perfil = [2400, 3000, 3700, 4400, 8000] # Teto dos perfis de renda respectivos às taxas nominais

perfil_renda_taxa = {'Taxa Nominal' : taxas_nominais,
                    'Perfil de Renda (até)' : renda_perfil}

# Criando um DataFrame para armazernar as informações do dicionário em uma Tabela.
df_taxas = pd.DataFrame(perfil_renda_taxa)

# Criando uma coluna com o valor das Taxas Efetivas utilizando a Nominal como base do cálculo
df_taxas['Taxa Efetiva'] = ((1 + df_taxas['Taxa Nominal'] / 12) ** 12 -1).round(6) 

# Reordenando a Tabela de acordo com a ordem desejada.
tabela_reordenada = ['Perfil de Renda (até)', 'Taxa Nominal', 'Taxa Efetiva']
df_taxas = df_taxas.reindex(columns=tabela_reordenada)

df_taxas

### 1.3.1 - Buscar na Web a informação atualizada da TR (taxa referencial) acumulada do mês anterior

In [ ]:
# Trecho de Código para Buscar o valor acumulado da TR de acordo o período informado

# Fazer a solicitação para a URL
url = 'https://www.debit.com.br/tabelas/tr-bacen'
response = requests.get(url)

# Verificar se a solicitação foi bem-sucedida
if response.status_code == 200:
    # Processar o conteúdo da página usando BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')
    
     # procurando o elemento <td> com o texto "mês/ano"
    elemento_procurado = soup.find('td', string=f'{mes_tr_str}/{ano_tr}')

    if elemento_procurado:
        # Encontre o próximo elemento <td> sem um atributo específico
        proximo_elemento = elemento_procurado.find_next('td', attrs={'data-v-789fe0da': True})

        if proximo_elemento:
            valor = proximo_elemento.text
            
valor = valor.replace(',', '.').replace('%', '')
taxa_referencial = (float(valor) /100) # Apresentar valor unitário
print('O Valor acumulado da TR do mês anterior é de:', taxa_referencial)


### 1.4 - Valores da Operação

In [ ]:
# Definir Valores da Operação

if meses > 0 and valor_renda_bruta <= 8000 and valor_venda <= 350000:
    
    financiamento_max = valor_venda * 0.8
    valor_entrada = valor_venda - (financiamento_max + desconto)
    
    if valor_entrada < 0:
        financiamento_max = financiamento_max + valor_entrada
        
    parcela_amortizacao = financiamento_max / meses
    
    taxa_administracao = 0
    if valor_renda_bruta > 4400:
        taxa_administracao = 25
        
    for i in range(len(renda_perfil)):
        
        if valor_renda_bruta <= renda_perfil[i]:
            juros_efetivos_a_a = taxa_referencial + df_taxas['Taxa Efetiva'][i]
            break
    
    juros_efetivos_a_m = ((1 + juros_efetivos_a_a)**(1/12)) -1
    
    # Definir Valor da Parcela
    pgto = npf.pmt(juros_efetivos_a_m, prazo_max, financiamento_max) * -1
    
    # Validador de Renda (Definido para saber se o cliente se adequa ao fluxo informado)
    if valor_renda_bruta >= (pgto * 100) /30: 
        validador = 'OK'
    else:
        validador = 'NOT'

else:
    validador = 'NOT'
    
    

### 1.5 Cálculo da Operação de Financiamento com parcelas mensais fixas* + seguros (PRICE TR)

In [ ]:
if meses > 0 and 'OK' in validador:
    
    # Definição inicial das listas e variáveis para utilizar no looping.
    
    saldo_total_fin = 0
    saldo_devedor = financiamento_max
    
    amortizacao_mais_juros = []
    seguro_tabela = []
    prestacao_encargos = []
    saldo_devedor_simulador = []
    administracao = []
    
    for i in range(meses):
        nascimento -= um_mes
        idade_futuro = total_dias / 365
        
        saldo_devedor -= parcela_amortizacao
        
        for j in range(len(idade_mip)):
            if idade_futuro <= idade_mip[j]:
                aliquota_mip = valor_aliquota[j]
                break
        
        valor_seguro = (saldo_devedor * aliquota_mip) + (saldo_devedor * seguro_dfi)
        #valor_seguro = round(valor_seguro, 2)
        
        if saldo_devedor >= 0:
            prestacao = pgto + valor_seguro + taxa_administracao
            #saldo_devedor = round(saldo_devedor, 2)
        else:
            valor_seguro = 0
            prestacao = pgto + valor_seguro + saldo_devedor + taxa_administracao
            saldo_devedor = 0
                
        #print('{} - A + J: R${:.2f} + Seguro: R${:.2f} | Prestação: R${:.2f} | Saldo Devedor: R${:.2f}'.format(
            #i+1, pgto, valor_seguro, prestacao, saldo_devedor))
        
        # Inserir as informações nas listas para gerar um DataFrame detalhado
        amortizacao_mais_juros.append(round(pgto, 2))
        seguro_tabela.append(round(valor_seguro, 2))
        prestacao_encargos.append(round(prestacao, 2))
        saldo_devedor_simulador.append(round(saldo_devedor, 2))
        administracao.append(round(taxa_administracao, 2))
        
        tabela_de_financiamento = {'Amortizacao + Juros' : amortizacao_mais_juros,
                                   'Seguros' : seguro_tabela,
                                   'Taxa de Administração' : taxa_administracao,
                                   'Encargos': prestacao_encargos,
                                   'Saldo Devedor':saldo_devedor_simulador }
        
        saldo_total_fin += prestacao
        
    df_tabela_fin = pd.DataFrame(tabela_de_financiamento)
    print('O Valor total acumulado ao longo de 360 meses foi de: R${:.2f}'.format(saldo_total_fin))
    
    if valor_entrada > 0:
        print('O Cliente ainda deverá dar uma entrada no valor de: R${:.2f}'.format(valor_entrada))

    # Salvar informações do DataFrame em uma Planilha Excel

    df_tabela_fin.to_excel('tabela_fin.xlsx', sheet_name='Financiamento', index=None)
    print('Uma planinha com o fluxo definido foi exportada com sucesso! =D')
        
else:
    print('''
    
    Não foi possível realizar o fluxo para financiar o imóvel...
    
    Verifique se:
    
    1 -  A renda do seu cliente comporta o valor informado.
    2 -  A renda do seu cliente está dentro do teto limite de até R$8.000,00.
    3 -  O valor do imóvel do seu cliente está dentro do teto limite de até R$350.000,00.
    4 -  O seu cliente tem idade para atingir o prazo mínimo do financiamento do imóvel.
    
    Obs: Caso o fluxo não se adeque às condições oferecidas pelo simulador, procure um especialista.
    
    ''')